## BRONZE TO SILVER LAYER

### Silver Layer - Stock Events History


In [ ]:
# Importing Common Utility Function
import re

import pandas as pd

from StockETL import GlobalPath

In [ ]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [ ]:
# Instantiate GlobalPath
stockdata_bronze_layer_path = GlobalPath("DATA/BRONZE/StockData")
stockevents_silver_file_path = GlobalPath(
    "DATA/SILVER/StockEvents/StockEvents_data.csv"
)
stockevents_silver_schema_file_path = GlobalPath(
    "CONFIG/DATA_CONTRACTS/SILVER/StockEvents.json"
)

### Data Processing

This section handles the collection of data from the Bronze layer.

- Files are checked for availability.
- Data is read from each file, processed, and stored in a list of DataFrames.
- Process stock event data from the Bronze layer and transform it for the Silver layer.


In [ ]:
# Initialize an empty list to store DataFrames
df_stock_events_list = []

# Generate file paths for available CSV files in the Bronze layer
file_paths = check_files_availability(stockdata_bronze_layer_path, file_pattern="*.csv")

# Process each CSV file found in the Bronze layer
for file_path in file_paths:
    print(f"Processing file => {file_path}")

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Extract the stock symbol by removing the date pattern from the file name
    df["symbol"] = re.sub(r"_\d{4}_\d{2}\.csv", "", file_path.name)

    # Append the DataFrame to the list
    df_stock_events_list.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(df_stock_events_list, ignore_index=True)

# Harmonize column names to ensure consistency
df = replace_punctuation_from_columns(df)

# Remove columns that contain only NA values
df.dropna(how="all", axis=1, inplace=True)

In [ ]:
# Convert datetime strings to date objects for consistency
df["date"] = pd.to_datetime(df["date"]).dt.date

# Pivot the DataFrame: columns become rows
df = df.melt(
    id_vars=["date", "symbol"],
    value_vars=["dividends", "stock_splits", "capital_gains"],
    var_name="event",
    value_name="value",
)

# Convert event names to uppercase for uniformity
df["event"] = df["event"].str.upper()

# Round numerical values to 2 decimal places
df["value"] = df["value"].fillna(0).round(2)

# Filter out rows where the value is zero or less than a threshold
df = df[df["value"] > 0]

# Align Datafame with DataContract
df = align_with_datacontract(df, stockevents_silver_schema_file_path)

# Save the cleaned and transformed DataFrame as a CSV file in the Silver layer
df.to_csv(stockevents_silver_file_path, index=False)
print("SILVER Layer CSV file for Stock Events history successfully created at =>")
print(stockevents_silver_file_path)

# Log the DataFrame debug for verification
df.info()